In [ ]:
import pandas as pd
# Load the datasets from CSV files
telecom_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GitHub/Data-weak2/Data/Copy of Week2_challenge_data_source(CSV).csv')
print("Telecom Data:")
print(telecom_df.head())

# Step 1: Identify the Top 10 Handsets Used by Customers (based on Handset Type)
top_handsets = telecom_df['Handset Type'].value_counts().head(10)
print("\nTop 10 Handsets:\n", top_handsets)

In [ ]:
print("Missing Values Before Cleaning:\n", telecom_df.isnull().sum())

In [ ]:
# Replace missing values with mean for numeric columns and mode for categorical columns
telecom_df['TCP DL Retrans. Vol (Bytes)'].fillna(telecom_df['TCP DL Retrans. Vol (Bytes)'].mean(), inplace=True)
telecom_df['TCP UL Retrans. Vol (Bytes)'].fillna(telecom_df['TCP UL Retrans. Vol (Bytes)'].mean(), inplace=True)
telecom_df['Avg RTT DL (ms)'].fillna(telecom_df['Avg RTT DL (ms)'].mean(), inplace=True)
telecom_df['Avg RTT UL (ms)'].fillna(telecom_df['Avg RTT UL (ms)'].mean(), inplace=True)
telecom_df['Avg Bearer TP DL (kbps)'].fillna(telecom_df['Avg Bearer TP DL (kbps)'].mean(), inplace=True)
telecom_df['Avg Bearer TP UL (kbps)'].fillna(telecom_df['Avg Bearer TP UL (kbps)'].mean(), inplace=True)
telecom_df['Handset Type'].fillna(telecom_df['Handset Type'].mode()[0], inplace=True)

In [ ]:
 #Outlier Detection and Handling
def replace_outliers_with_mean(column):
    mean = column.mean()
    std_dev = column.std()
    lower_bound = mean - 3 * std_dev
    upper_bound = mean + 3 * std_dev
    column = np.where(column < lower_bound, mean, column)
    column = np.where(column > upper_bound, mean, column)
    return column

telecom_df['Avg RTT DL (ms)'] = replace_outliers_with_mean(telecom_df['Avg RTT DL (ms)'])
telecom_df['Avg RTT UL (ms)'] = replace_outliers_with_mean(telecom_df['Avg RTT UL (ms)'])
telecom_df['Avg Bearer TP DL (kbps)'] = replace_outliers_with_mean(telecom_df['Avg Bearer TP DL (kbps)'])
telecom_df['Avg Bearer TP UL (kbps)'] = replace_outliers_with_mean(telecom_df['Avg Bearer TP UL (kbps)'])

In [ ]:
# Step 4: Aggregation
customer_aggregation = telecom_df.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]  # Most common handset type
}).reset_index()

In [ ]:
# Average TCP Retransmission and Throughput
customer_aggregation['Average TCP Retransmission'] = (
    customer_aggregation['TCP DL Retrans. Vol (Bytes)'] + 
    customer_aggregation['TCP UL Retrans. Vol (Bytes)']
) / 2

In [ ]:
customer_aggregation['Average Throughput'] = (
    customer_aggregation['Avg Bearer TP DL (kbps)'] + 
    customer_aggregation['Avg Bearer TP UL (kbps)']
) / 2

In [ ]:
# Drop unnecessary columns
customer_aggregation.drop(columns=['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)', 
                                    'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)'], inplace=True)

In [ ]:
 # Rename columns for clarity
customer_aggregation.rename(columns={
    'Avg RTT DL (ms)': 'Average RTT DL (ms)',
    'Avg RTT UL (ms)': 'Average RTT UL (ms)'
}, inplace=True)

In [ ]:
print("\nAggregated Customer Data:\n", customer_aggregation.head())